In [ ]:
import pandas as pd
import logging
import random

logger = logging.getLogger(__name__)
dr = '/home/egk204/PycharmProjects/code-trans-aug/'
data = pd.read_pickle(dr+'storage/data_original.pkl')

#collection of parsable statements from original dataset for random insertion

o = data
ja = o[o.language_extension=='java']
py = o[o.language_extension=='py']

px = []
for i in py.code:
    px.extend(i.split('\n'))

import ast
def check_py_ast(c):
    try:
        ast.parse(c)
        return True
    except:
        return False
    
pxp = [s for s in px if check_py_ast(s) ]

jx = []
for i in ja.code:
    jx.extend(i.split(';'))

import javalang
def check_ja_ast(c):
    try:
        tokens = javalang.tokenizer.tokenize(c+";")
        parser = javalang.parser.Parser(tokens)
        parser.parse_expression()
        return True
    except:
        return False
    
jxp = [s for s in jx if check_ja_ast(s) ]

In [ ]:
import random
class common_transform:
    def transform_copy_statement(self):
        self.pos = random.randrange(len(self.splitted) if len(self.splitted)>0 else 1 )
        self.splitted.insert(self.pos, self.pos-1)
        return self.splitted
    
    def transform_delete_statement(self):
        if len(self.splitted)>0:
            self.pos = random.randrange(len(self.splitted) if len(self.splitted)>0 else 1 )
            self.splitted.pop(self.pos)
        return self.splitted
    def transform_swap_statement(self):
        if len(self.splitted)>=2:
            self.pos = random.randrange(len(self.splitted) if len(self.splitted)>0 else 1 )
            l = self.splitted
            pos1 = self.pos
            if self.pos == len(self.splitted)-1:
                pos2 = self.pos-1
            else:
                pos2 = self.pos+1
            l[pos1], l[pos2] = l[pos2], l[pos1]
            self.splitted = l
        return self.splitted
    def transform_replace_op(self):
        if len(self.splitted)>0:
            self.pos = random.randrange(len(self.splitted) if len(self.splitted)>0 else 1 )
            s = str(self.splitted[self.pos])
            ops = ['+', '-', '*', '/', '>', '<', '<=', '>=','-=','+=']
            d = {'+':'-', 
                '-':'+', 
                '*':'/', 
                '/':'*', 
                '>':'<', 
                '<':'>', 
                '<=':'>=', 
                '>=':'+=',
                '-=':'+=',
                '+=':'-='         
            }
            p1 = ops[random.randrange(len(ops))]
            p2 = d[p1]
            s = s.replace(p1, p2)
            self.splitted[self.pos] = s
        return self.splitted
    
    
    
class python_transform(common_transform):
    def __init__(self, code):
        self.code = code
        self.splitted = code.split('\n')
        self.pos = random.randrange(len(self.splitted) if len(self.splitted)>0 else 1 )
    def transform_insert_statement(self):
        self.pos = random.randrange(len(self.splitted) if len(self.splitted)>0 else 1 )
        #sample statements
        statements = [
            'a = 10', 
            'try:\nprint(x)\nexcept:\nprint("An exception occurred")',
            'myNumbers = {1, 2, 3};',
            'pos1, pos2  = 1, 3'
            
        ]
        #self.splitted.insert(self.pos, statements[random.randrange(len(statements))])
        self.splitted.insert(self.pos, random.choice(pxp))
        return self.splitted
    def transform_insert_comment(self):
        self.pos = random.randrange(len(self.splitted) if len(self.splitted)>0 else 1 )
        self.splitted.insert(self.pos, '#this is a comment;')
        return self.splitted


class java_transform(common_transform):
    def __init__(self, code):
        self.code = code
        self.splitted = code.split(';')
        self.pos = random.randrange(len(self.splitted) if len(self.splitted)>0 else 1 ) 
    #sample statements    
    def transform_insert_statement(self):
        self.pos = random.randrange(len(self.splitted) if len(self.splitted)>0 else 1 )
        #sample
        statements =[
            'int a = 10;', 
            'try {//  Block of code to try } catch(Exception e) { //  Block of code to handle errors}',
            'int[] myNumbers = {1, 2, 3};',
            'System.out.println(myNumbers[10]);',  
        ]
        
        #self.splitted.insert(self.pos, statements[random.randrange(len(statements))])
        self.splitted.insert(self.pos, random.choice(jxp))
        return self.splitted
    
    def transform_insert_comment(self):
        self.pos = random.randrange(len(self.splitted) if len(self.splitted)>0 else 1 )
        self.splitted.insert(self.pos, '//this is a comment;')
        return self.splitted
    

def transform(code, ext):
    if ext =='java':
        obj = java_transform(code)
    if ext =='py':
        obj = python_transform(code)
    #print(obj.code, obj.splitted, obj.pos)
    
    #select any transformation
    for i in range(0,random.randrange(6)):
        options = [0,1,2,3,4,5]
        rand_func = random.choice(options)
        if rand_func == 0:
            c = obj.transform_insert_statement()
        elif rand_func == 1:
            c = obj.transform_delete_statement()
        elif rand_func == 2:
            c = obj.transform_insert_comment()
        elif rand_func == 3:
            c = obj.transform_swap_statement()
        elif rand_func == 4:
            c = obj.transform_replace_op()
        elif rand_func == 5:
            c = obj.transform_copy_statement()
    c = obj.splitted
    #print(type(c))
    return '\n'.join(str(i) for i in c) if ext == 'py' else ';'.join(str(i) for i in c)

In [ ]:
aug = []
for index, row in data.iterrows():
    code_ = row.code
    lang_ = row.language_extension
    print(row.id)
    c_ = transform(code_, lang_)
    aug.append(list(row[:2])+[c_]+list(row[3:]))

In [ ]:
aug = pd.DataFrame(aug, columns=data.columns)

In [ ]:
aug.id = aug.id+38110

In [ ]:
aug

In [ ]:
#aug.to_pickle(dr+'storage/alternative/data_alternative.pkl')

In [ ]:
import pandas as pd
dr = '/home/egk204/PycharmProjects/code-trans-aug/'
dra = dr+'storage/alternative/'

data = pd.read_pickle(dr+'storage/data_original.pkl')
aug = pd.read_pickle(dr+'storage/alternative/data_alternative.pkl')

org_java = data[data.language_extension=='java']
org_py = data[data.language_extension=='py'] #for GMN i have converted in transpiled
aug_java = aug[aug.language_extension=='java']
aug_py = aug[aug.language_extension=='py'] #for GMN needs to convert


In [ ]:
aug[['id','language_extension', 'code']].to_csv(dr+'storage/alternative/aug_alt.csv', index=False, header=None)

In [ ]:
merged = pd.concat([data,aug])
mapping = {item:i+1 for i, item in enumerate(merged["problem_class"].unique())}
merged["task"] = merged['problem_class'].apply(lambda x: mapping[x])
merged.to_pickle(dr+'storage/alternative/codes_alt.pkl')

In [ ]:
merged

In [ ]:
#clon-non clone pair java - py -large

In [ ]:
alt_aug_java = merged[merged.language_extension=='java']
alt_aug_py = merged[merged.language_extension=='py']

In [ ]:
def  filter_for_pair (df):
    df = df[['id', 'task']]
    return df

In [ ]:
filter_for_pair(alt_aug_java).to_csv(dr+'storage/alternative/alt_aug_java_large.csv', header=None, index=False)
filter_for_pair(alt_aug_py).to_csv(dr+'storage/alternative/alt_aug_py_large.csv', header=None, index=False)

In [ ]:
def filter_code(df):
    cp = 1
    cj = 1
    refined = []
    for index, row in df.iterrows():
        if row.language_extension =='py':
            if(len(row.code.split('\n'))>6):
                cp+=1
                refined.append(list(row))
        if row.language_extension =='java':
            if(len(row.code.split(';'))>6):
                cj+=1
                refined.append(list(row))
    print(df[df.language_extension =='py'].shape[0]+ df[df.language_extension =='java'].shape[0], pd.DataFrame(refined).shape, (cp+cj)/df.shape[0])
    return pd.DataFrame(refined, columns=list(df.columns))

In [ ]:
merged_small = filter_code(merged)

In [ ]:
merged_small

In [ ]:
alt_aug_java_small = merged_small[merged_small.language_extension=='java']
alt_aug_py_small = merged_small[merged_small.language_extension=='py']

In [ ]:
filter_for_pair(alt_aug_java_small).to_csv(dr+'storage/alternative/alt_aug_java_small.csv', header=None, index=False)
filter_for_pair(alt_aug_py_small).to_csv(dr+'storage/alternative/alt_aug_py_small.csv', header=None, index=False)

In [ ]:
#create dataset
import pandas as pd
dr = '~/PycharmProjects/code-trans-aug/'
dra = dr+'storage/alternative/'

cols = ['id1', 'id2', 'label']
clones_sm =pd.read_csv(dr+'storage/alternative/clones_augmented_small_sample.csv', names=cols)
print(clones_sm.shape)
non_clones_sm =pd.read_csv(dr+'storage/alternative/non_clones_augmented_small_sample.csv', names=cols).sample(clones_sm.shape[0])
print(non_clones_sm.shape)
da = pd.concat([clones_sm, non_clones_sm]).sample(frac=.6)

#small
do = pd.read_csv(dr+'/storage/dataset_small/dataset_org.csv', names=cols)#read original small
dataset_or_80_20_aug = do.append(da.sample(int(len(do)*0.25)))

In [ ]:
dataset_or_80_20_aug.to_csv(dr+'storage/alternative/alt_small_dataset_or_80_20_aug.csv', header=None, index=False)

In [ ]:
#large #shuf -n
import pandas as pd
dr = '~/PycharmProjects/code-trans-aug/'

cols = ['id1', 'id2', 'label']
clones =pd.read_csv(dr+'storage/alternative/clones_augmented_large_sample.csv', names=cols)
print(clones.shape)
non_clones =pd.read_csv(dr+'storage/alternative/non_clones_augmented_large_sample.csv', names=cols).sample(clones.shape[0])
print(non_clones.shape)

da = clones.append(non_clones)

do = pd.read_csv(dr+'/storage/dataset_large/dataset_org.csv', names=cols)#read original large


In [ ]:
dataset_or_80_20_aug = do.append(da.sample(int(len(do)*0.25)))


In [ ]:
dataset_or_80_20_aug

In [ ]:
dataset_or_80_20_aug.to_csv(dr+'storage/alternative/alt_large_dataset_or_80_20_aug.csv', header=None, index=False)

In [ ]:
import sys
sys.path.append("~/PycharmProjects/code-trans-aug/")
import pandas as pd
storage_dir = '~/PycharmProjects/code-trans-aug/storage/'
sv_dir = '~/PycharmProjects/code-trans-aug/storage/alternative/'
import pandas as pandas

In [ ]:
#ensure ANTLR features for this step

In [ ]:
import pandas as pd

aug = pd.read_csv('aug_alt.csv', names=['id', 'lang', 'code'])
for ind, row in aug.iterrows():
    fi = 'aug/'+str(row.id) +'.'+ str(row.lang)
    with open(fi, 'w') as f:
        f.write(str(row.code))
        
s = '~/PycharmProjects/code-trans-aug/storage/'
column_names = ['fpath','variable', 'arguments', 'operators', 'expressions', 'loops','operands', 'exceptions', 'exception_clause', 'mccable_complex']
j = pd.read_csv(s+'feat_alt/java_features.csv', names=column_names)
p = pd.read_csv(s+'feat_alt/python_features.csv', names=column_names)
j['id']=[f.split('/')[-1:][0].split('.')[0]  for f in j.fpath]
p['id']=[f.split('/')[-1:][0].split('.')[0]  for f in p.fpath]
pd.concat([j.iloc[:,1:], p.iloc[:,1:]]).to_csv(s+'alternative/alt_feat.csv', index=None)

In [ ]:
def feats_to_dict(dft, feats_dict):
    
    col = ['variable', 'arguments', 'operators', 'expressions', 'loops','operands', 'exceptions', 'exception_clause', 'mccable_complex']
    for ind, row in dft.iterrows():
        key = int(row.id)
        value = list(row[col])
        feats_dict[key] = value
    return feats_dict
def get_antlr_feature_dictionary(storage_dir = storage_dir):
    # load features 
    #generate dict   
    column_names = ['variable', 'arguments', 'operators', 'expressions', 'loops','operands', 'exceptions', 'exception_clause', 'mccable_complex', 'file_path', 'problem_class','id']
    
    alt = pd.read_csv(storage_dir+'alternative/alt_feat.csv').sort_values(['id'])
    ojf = pd.read_csv(storage_dir+'feat_original/java_features_pc_id.csv', names=column_names)
    opf = pd.read_csv(storage_dir+'feat_original/python_features_pc_id.csv', names=column_names)

    feats_dict = {}
    feats_dfs = [alt, opf, ojf]
    for dft in feats_dfs:
        feats_dict = feats_to_dict(dft, feats_dict)
    print('features dict len:',len(feats_dict))

    return feats_dict

fd = get_antlr_feature_dictionary()

In [ ]:
def get_dataset(df, data_dict):
    print('printing data shape, data_dict length',df.shape, len(data_dict))
    dataset = []
    n = [0]*9
    #for df in [clones, non_clones]:
    for ind in range (len(df)):
        row = list(df.iloc[ind,:])
        if data_dict.get(row[0]) is not None:
            x1 = data_dict.get(row[0])
        else: 
            x1 = n
        if data_dict.get(row[1]) is not None:
            x2 =  data_dict.get(row[1])
        else:
            x2 =n
        label = row[2] 
        dataset.append(x1+x2+[label])
    
    d = pd.DataFrame(dataset)
    d = d.fillna(0)
    return d

pairs = pd.read_csv(sv_dir+'alt_small_dataset_or_80_20_aug.csv', header=None)
dset = get_dataset(pairs, fd)
dset = dset[dset.values.sum(axis=1) != 0] 
dset.to_csv(sv_dir+'alt_ft_lcdsa_small.csv', header=None, index=False)
pairs = pd.read_csv(sv_dir+'alt_large_dataset_or_80_20_aug.csv', header=None)
dset = get_dataset(pairs, fd)
dset = dset[dset.values.sum(axis=1) != 0] 
dset.to_csv(sv_dir+'alt_ft_clcdsa_large.csv', header=None, index=False)



In [ ]:
#c4


In [ ]:
import pandas as pd

In [ ]:
pd.read_pickle('/home/egk204/PycharmProjects/code-trans-aug/storage/alternative/data_alternative.pkl')

In [ ]:
def split_dataset(data):
    ratios = [8,1,1]

    data_num = len(data)
    train_split = int(ratios[0] / sum(ratios) * data_num)
    val_split = train_split + int(ratios[1] / sum(ratios) * data_num)

    data = data.sample(frac=1, random_state=666)
    
    train_list = data.iloc[:train_split]
    valid_list = data.iloc[train_split:val_split]
    test_list = data.iloc[val_split:]
    
    return train_list, valid_list, test_list


In [ ]:
dr = '~/PycharmProjects/code-trans-aug/storage/alternative/'
d_t = 'c4/'
cols = ['id1', 'id2', 'label']
import pandas as pd

In [ ]:
augmented_pairs = pd.read_csv(dr+'alt_large_dataset_or_80_20_aug.csv', names=cols)
large = augmented_pairs[augmented_pairs.label==1]

In [ ]:
large

In [ ]:
train, dev, test = split_dataset(large)
train.to_csv(dr+d_t+'train_large.csv',header=None, index=False)

test.to_csv(dr+d_t+'test_large.csv',header=None, index=False)

dev.to_csv(dr+d_t+'dev_large.csv',header=None, index=False)


In [ ]:
augmented_pairs = pd.read_csv(dr+'alt_small_dataset_or_80_20_aug.csv', names=cols)
small = augmented_pairs[augmented_pairs.label==1]

In [ ]:
train, dev, test = split_dataset(small)
train.to_csv(dr+d_t+'train_small.csv',header=None, index=False)

test.to_csv(dr+d_t+'test_small.csv',header=None, index=False)

dev.to_csv(dr+d_t+'dev_small.csv',header=None, index=False)

In [ ]:
small

In [ ]:
train

In [ ]:
#need to make GMN converted
dr = '/home/egk204/PycharmProjects/code-trans-aug/'
gmn_original = pd.read_pickle(dr+'storage/dataset_large/gmn_original_final.pkl')
gmn_trans_alt = pd.read_pickle(dr+'storage/alternative/alternative_to_java.pkl') #will replace with translated one



In [ ]:
mapping = {item:i+1 for i, item in enumerate(gmn_trans_alt["problem_class"].unique())}

In [ ]:
gmn = pd.concat([gmn_original, gmn_trans_alt])
gmn["task"] = gmn['problem_class'].apply(lambda x: mapping[x])

In [ ]:
gmn

In [ ]:
gmn.to_pickle(dr+'storage/alternative/gmn_codes_alt.pkl')